In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [2]:
!wget -q -O 'bus_stop_data.json' https://opendata.arcgis.com/datasets/b595b1365f8e47618658692539d946de_1.geojson
print('Data downloaded!')

Data downloaded!


In [3]:
with open('bus_stop_data.json') as json_data:
    bus_stop_data = json.load(json_data)

In [4]:
bus_stop = bus_stop_data['features']

In [5]:
bus_stop[0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1006840,
  'ProjectID': 'CHCMETRO',
  'ProjectName': 'Metro Christchurch',
  'PlatformTag': 1984,
  'PlatformNo': 37616,
  'PlatformName': 'Frankleigh St near Alvarez Pl',
  'RoadName': 'Frankleigh St',
  'BearingToRoad': 286,
  'RouteNos': '120|60|Or',
  'RouteTags': '130|241|299',
  'Routes': '120:241|60:130|Or:299',
  'NZMGX': None,
  'NZMGY': None,
  'NZMS260': '',
  'NZTMX': None,
  'NZTMY': None,
  'NZTOPO50': None,
  'WGS84_LONGITUDE': 172.6194731,
  'WGS84_LATITUDE': -43.55977629,
  'SOURCECODE': 'GPS',
  'QARCODE': 1,
  'CREATEDBY': 'MetroInfo',
  'CREATEDDATE': '2010-12-10T14:04:23Z',
  'MODIFIEDBY': None,
  'MODIFIEDDATE': '2020-05-07T04:31:03Z'},
 'geometry': {'type': 'Point',
  'coordinates': [172.61947310000016, -43.55977628999994]}}

In [23]:
bus_stop[0]

{'type': 'Feature',
 'properties': {'OBJECTID': 1006840,
  'ProjectID': 'CHCMETRO',
  'ProjectName': 'Metro Christchurch',
  'PlatformTag': 1984,
  'PlatformNo': 37616,
  'PlatformName': 'Frankleigh St near Alvarez Pl',
  'RoadName': 'Frankleigh St',
  'BearingToRoad': 286,
  'RouteNos': '120|60|Or',
  'RouteTags': '130|241|299',
  'Routes': '120:241|60:130|Or:299',
  'NZMGX': None,
  'NZMGY': None,
  'NZMS260': '',
  'NZTMX': None,
  'NZTMY': None,
  'NZTOPO50': None,
  'WGS84_LONGITUDE': 172.6194731,
  'WGS84_LATITUDE': -43.55977629,
  'SOURCECODE': 'GPS',
  'QARCODE': 1,
  'CREATEDBY': 'MetroInfo',
  'CREATEDDATE': '2010-12-10T14:04:23Z',
  'MODIFIEDBY': None,
  'MODIFIEDDATE': '2020-05-07T04:31:03Z'},
 'geometry': {'type': 'Point',
  'coordinates': [172.61947310000016, -43.55977628999994]}}

In [6]:
# define the dataframe columns
column_names = ['StopName', 'RoadName', 'ProjectName','Latitude', 'Longitude'] 

# instantiate the dataframe
filtered_bus_stop = pd.DataFrame(columns=column_names)

In [7]:
filtered_bus_stop

,StopName,RoadName,ProjectName,Latitude,Longitude


In [8]:
for data in bus_stop:
    SN = data['properties']['PlatformName'] 
    RN = data['properties']['RoadName']
    PN = data['properties']['ProjectName']
        
    latlon = data['geometry']['coordinates']
    lat = latlon[1]
    lon = latlon[0]
    
    filtered_bus_stop = filtered_bus_stop.append({'StopName': SN,
                                                  'RoadName': RN,
                                                  'ProjectName':PN,
                                                   'Latitude': lat,
                                                   'Longitude': lon}, ignore_index=True)

In [9]:
filtered_bus_stop.head()

,StopName,RoadName,ProjectName,Latitude,Longitude
0,Frankleigh St near Alvarez Pl,Frankleigh St,Metro Christchurch,-43.559776,172.619473
1,Ti Rakau Dr near Delta Wy,Ti Rakau Dr,Metro Christchurch,-43.551737,172.694984
2,Hoon Hay Rd near Maryhill Ave,Hoon Hay Rd,Metro Christchurch,-43.562218,172.607377
3,Memorial Ave near Grahams Rd,Memorial Ave,Metro Christchurch,-43.506651,172.571503
4,Aldwins Rd near Marlborough St,Aldwins Rd,Metro Christchurch,-43.534654,172.671560


In [10]:
df_bus_stop = filtered_bus_stop[filtered_bus_stop['ProjectName'].str.contains('Christchurch', regex=False)]

In [12]:
df_bus_stop.head()

,StopName,RoadName,ProjectName,Latitude,Longitude
0,Frankleigh St near Alvarez Pl,Frankleigh St,Metro Christchurch,-43.559776,172.619473
1,Ti Rakau Dr near Delta Wy,Ti Rakau Dr,Metro Christchurch,-43.551737,172.694984
2,Hoon Hay Rd near Maryhill Ave,Hoon Hay Rd,Metro Christchurch,-43.562218,172.607377
3,Memorial Ave near Grahams Rd,Memorial Ave,Metro Christchurch,-43.506651,172.571503
4,Aldwins Rd near Marlborough St,Aldwins Rd,Metro Christchurch,-43.534654,172.671560


In [14]:
address = 'Christchurch, New Zealand'

geolocator = Nominatim(user_agent="chch_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Christchurch, New Zealand are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Christchurch, New Zealand are -43.530955, 172.6366455.


In [15]:
map_chch = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, sn, rn in zip(df_bus_stop['Latitude'], df_bus_stop['Longitude'], df_bus_stop['StopName'], df_bus_stop['RoadName']):
    label = '{}, {}'.format(sn, rn)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chch)  
    
map_chch

In [16]:
k=5
bus_stop_clustering = df_bus_stop.drop(['StopName','RoadName','ProjectName'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(bus_stop_clustering)
kmeans.labels_
df_bus_stop.insert(0, 'Cluster Labels', kmeans.labels_)

In [19]:
df_bus_stop.head()

,Cluster Labels,StopName,RoadName,ProjectName,Latitude,Longitude
0,3,Frankleigh St near Alvarez Pl,Frankleigh St,Metro Christchurch,-43.559776,172.619473
1,1,Ti Rakau Dr near Delta Wy,Ti Rakau Dr,Metro Christchurch,-43.551737,172.694984
2,3,Hoon Hay Rd near Maryhill Ave,Hoon Hay Rd,Metro Christchurch,-43.562218,172.607377
3,0,Memorial Ave near Grahams Rd,Memorial Ave,Metro Christchurch,-43.506651,172.571503
4,1,Aldwins Rd near Marlborough St,Aldwins Rd,Metro Christchurch,-43.534654,172.671560


In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, sn, cluster in zip(df_bus_stop['Latitude'], df_bus_stop['Longitude'], df_bus_stop['StopName'], df_bus_stop['Cluster Labels']):
    label = folium.Popup(str(sn) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

This notebook is part of a course on **Coursera** called _Applied Data Science Capstone_. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By    | Change Description         |
| ----------------- | ------- | ------------- | -------------------------- |
| 2020-11-26        | 2.0     | Lakshmi Holla | Updated the markdown cells |
|                   |         |               |                            |
|                   |         |               |                            |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
